In [42]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

import gensim
from gensim import models, corpora, similarities
from gensim.models import LsiModel
from gensim.utils import lemmatize
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_punctuation, strip_tags,preprocess_string
from gensim.parsing.preprocessing import strip_numeric, strip_short, strip_multiple_whitespaces, strip_non_alphanum
from gensim.models.coherencemodel import CoherenceModel

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords





In [43]:
df_corpus = pd.read_csv('bbc-text.csv',names=['category', 'text'])
df_corpus.head()


,category,text
0,category,text
1,tech,tv future in the hands of viewers with home th...
2,business,worldcom boss left books alone former worldc...
3,sport,tigers wary of farrell gamble leicester say ...
4,sport,yeading face newcastle in fa cup premiership s...


In [44]:
corpus = df_corpus['text'].values.tolist()
corpus[:5]

['text',
 'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-

In [45]:
corpus = corpus[1:]


In [46]:
my_filter = [lambda x: x.lower(), strip_tags, strip_punctuation,
                                  strip_multiple_whitespaces, strip_numeric,
                                  remove_stopwords, strip_short, stem_text
            ]

In [47]:
def preprocessing(corpus):
    for document in corpus:
        doc = strip_numeric(document)
        doc = remove_stopwords(doc)
        doc = strip_short(doc,3)
        #doc = stem_text(doc)
        doc = strip_punctuation(doc)
        strip_tags(doc)
        yield gensim.utils.tokenize(doc, lower=True)


In [48]:
texts = preprocessing(corpus)


In [49]:
dictionary = corpora.Dictionary(texts)


In [50]:
dictionary.filter_extremes(no_below=1, keep_n=25000)


In [51]:
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in preprocessing(corpus)]

In [52]:
tfidf = models.TfidfModel(doc_term_matrix)

In [53]:
corpus_tfidf = tfidf[doc_term_matrix]

In [54]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary)  # initialize an LSI transformation

In [55]:
pprint(lsi.print_topics(num_topics=5, num_words=5))

[(0,
  '0.137*"labour" + 0.123*"election" + 0.118*"blair" + 0.110*"brown" + '
  '0.096*"government"'),
 (1,
  '-0.286*"labour" + -0.250*"election" + -0.239*"blair" + -0.218*"brown" + '
  '-0.178*"party"'),
 (2,
  '-0.148*"england" + 0.146*"mobile" + 0.116*"growth" + 0.111*"bn" + '
  '-0.102*"wales"'),
 (3,
  '-0.314*"film" + -0.174*"best" + -0.151*"awards" + 0.149*"england" + '
  '-0.125*"award"'),
 (4,
  '0.228*"film" + -0.185*"mobile" + 0.147*"best" + 0.132*"bn" + '
  '0.131*"dollar"')]
